<a href="https://colab.research.google.com/github/jbarrasa/goingmeta/blob/main/session25/python/GM_25_LLMs_for_automated_KG_construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j openai pandas rdflib-neo4j

In [ ]:
import os, time
from pprint import pprint
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['NEO4J_URL'] = userdata.get('NEO4J_URL')
os.environ['NEO4J_USR'] = userdata.get('NEO4J_USR')
os.environ['NEO4J_PWD'] = userdata.get('NEO4J_PWD')

url = os.environ['NEO4J_URL']
username = os.environ['NEO4J_USR']
password = os.environ['NEO4J_PWD']

#Explore the croissant metadata (it's JSON-LD)

In [30]:
# croassant_file_path = 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/croissants/metadata-nyhousing.json'
# data_file_path = 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/csvs/NY-House-Dataset.csv'

croassant_file_path = 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/croissants/metadata-supplychain.json'
data_file_path = 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/csvs/SCMS_Delivery_History_Dataset_20150929.csv'

In [31]:
from rdflib_neo4j import Neo4jStoreConfig, Neo4jStore, HANDLE_VOCAB_URI_STRATEGY
from rdflib import Graph

auth_data = {'uri': url,
             'database': "neo4j",
             'user': username,
             'pwd': password}

config = Neo4jStoreConfig(auth_data=auth_data,
                          handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
                          batching=True)

graph_store = Graph(store=Neo4jStore(config=config))

#temporary hack to parse JSON-LD until new version of rdflib-neo4j released
temp = Graph()
temp.parse(croassant_file_path, format="json-ld")

graph_store.parse(data=temp.serialize(format="ttl"), format="ttl")
graph_store.close(True)

Uniqueness constraint on :Resource(uri) is created.
IMPORTED 288 TRIPLES


# Explore the metadata graph

```
match graph = (:Dataset)-[:recordSet]->(:RecordSet)-[:field]->(:Field)-[:dataType]->()
return graph
```

# Get a textual description to use when prompting the LLM

In [32]:
from neo4j import GraphDatabase

with GraphDatabase.driver(url, auth=(username, password)) as driver:
    driver.verify_connectivity()

    features_query = """
    match (d:Dataset)-[:recordSet]->(rs:RecordSet)-[:field]->(f:Field)-[:dataType]->(dt),
          (f)-[:source]->()-[:extract]->(e)
    return d.name as datasetname, d.description as datasetdescription,
           collect ({featurename : e.column, datatype : dt.uri }) as datasetfeatures
    """
    records, summary, keys = driver.execute_query(features_query)

    result = records[0]

    ds_name = result['datasetname']
    ds_description = result['datasetdescription']
    ds_features = result['datasetfeatures']


# Prompt creation

In [33]:


system = "You are a data modelling expert capable of creating high quality entity-relationship models from flat datasets"

prompt=f"""

From the list of features in the following dataset create a list of entities and relationships with their
attributes in a simple json format and map them to the features in the dataset.

The attributes don't need to be named after the features in the dataset, but they should be mapped to the corresponding feature name.
No extra text or comments, only the json as output.

DATASET NAME: {ds_name}

DATASET DESCRIPTION: {ds_description}

DATASET FEATURES: {ds_features}
"""

print(prompt)



From the list of features in the following dataset create a list of entities and relationships with their
attributes in a simple json format and map them to the features in the dataset.

The attributes don't need to be named after the features in the dataset, but they should be mapped to the corresponding feature name.
No extra text or comments, only the json as output.

DATASET NAME: Supply-Chain-Shipment-Pricing-Data

DATASET DESCRIPTION: This data set provides supply chain health commodity shipment and pricing data.

DATASET FEATURES: [{'featurename': 'Delivered to Client Date', 'datatype': 'https://schema.org/Date'}, {'featurename': 'Molecule/Test Type', 'datatype': 'https://schema.org/Text'}, {'featurename': 'Dosage', 'datatype': 'https://schema.org/Text'}, {'featurename': 'ASN/DN #', 'datatype': 'https://schema.org/Text'}, {'featurename': 'Line Item Quantity', 'datatype': 'https://schema.org/Integer'}, {'featurename': 'Brand', 'datatype': 'https://schema.org/Text'}, {'featurena

# Call the completions API

In [34]:
from openai import OpenAI

client = OpenAI()
completion = client.chat.completions.create(model="gpt-4",
                                                  temperature=0,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ])



In [35]:
genschema = completion.choices[0].message.content
print(genschema)

{
  "entities": [
    {
      "name": "Shipment",
      "attributes": [
        {"name": "ShipmentID", "type": "https://schema.org/Text", "mappedTo": "ID"},
        {"name": "DeliveryDate", "type": "https://schema.org/Date", "mappedTo": "Delivered to Client Date"},
        {"name": "ScheduledDeliveryDate", "type": "https://schema.org/Date", "mappedTo": "Scheduled Delivery Date"},
        {"name": "RecordedDeliveryDate", "type": "https://schema.org/Date", "mappedTo": "Delivery Recorded Date"},
        {"name": "ShipmentMode", "type": "https://schema.org/Text", "mappedTo": "Shipment Mode"},
        {"name": "FulfillVia", "type": "https://schema.org/Text", "mappedTo": "Fulfill Via"},
        {"name": "FreightCost", "type": "https://schema.org/Text", "mappedTo": "Freight Cost (USD)"}
      ]
    },
    {
      "name": "Product",
      "attributes": [
        {"name": "ProductName", "type": "https://schema.org/Text", "mappedTo": "Item Description"},
        {"name": "Brand", "type": "https:

# Data import script generators (generic)

In [36]:
# adjust based on the JSON generated by the LLM
# there are beter (and more robust) ways to do this ;-)

e_name = "name"
a_name = "name"
map_att = "mappedTo"
r_name = "type"
r_from = "from"
r_to = "to"


def getIdValExprFor(sch, ename):
  result = []
  for x in sch["entities"]:
    if x[e_name] == ename:
      for y in x["attributes"]:
        result.append("toString(record.`" + y[map_att] + "`)")
  return " apoc.hashing.fingerprint( " + " + ".join(result) + " )"



def generateImportScript(sch):

  constraints = {}
  cypher_import = {}

  for e in sch["entities"]:
      constraints[e[e_name]] = "CREATE CONSTRAINT IF NOT EXISTS FOR (n:" + e[e_name] + ") REQUIRE n._id IS UNIQUE; "
      cypher = []
      cypher.append("unwind $records AS record")
      cypher.append("merge (n:`" + e[e_name] + "` { _id: " +  getIdValExprFor(sch, e[e_name]) + "} )")
      for p in e["attributes"]:
        propName = p[a_name]
        #type conversion ?
        #propType = p["datatype"]
        cypher.append("set n.`" + propName + "` = record.`" + p[map_att] + "`")
      cypher.append("return count(*) as total ;")
      cypher_import[e[e_name]] = ' \n'.join(cypher)

  for r in sch["relationships"]:
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("match (source:" + r[r_from] + " { _id : " + getIdValExprFor(sch, r[r_from]) + "} )")
    cypher.append("match (target:" + r[r_to] + " { _id : " + getIdValExprFor(sch, r[r_to]) + "} )")
    cypher.append("merge (source)-[r:`"+ r[r_name] +"`]->(target)")
    cypher.append("return count(*) as total")
    cypher_import[r[r_name]] = ' \n'.join(cypher)

  return cypher_import , constraints

# Run the script generators on the model+mapping produced by the LLM

In [37]:
cypher_import, constraints = generateImportScript(eval(genschema))

In [38]:
print("\n\n#CONSTRAINTS:\n")
for q in constraints.keys():
  print(q + ": \n"+ constraints[q] + "\n")

print("#\n\nLOADERS:\n")
for q in cypher_import.keys():
  print(q + ": \n\nfile: " + data_file_path + "\n\n"+ cypher_import[q] + "\n\n")



#CONSTRAINTS:

Shipment: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Shipment) REQUIRE n._id IS UNIQUE; 

Product: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Product) REQUIRE n._id IS UNIQUE; 

Vendor: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Vendor) REQUIRE n._id IS UNIQUE; 

Order: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Order) REQUIRE n._id IS UNIQUE; 

LineItem: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:LineItem) REQUIRE n._id IS UNIQUE; 

ManufacturingSite: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:ManufacturingSite) REQUIRE n._id IS UNIQUE; 

Country: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Country) REQUIRE n._id IS UNIQUE; 

#

LOADERS:

Shipment: 

file: https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/csvs/SCMS_Delivery_History_Dataset_20150929.csv

unwind $records AS record 
merge (n:`Shipment` { _id:  apoc.hashing.fingerprint( toString(record.`ID`) + toString(record.`Delivered to Client Date`) + toString(record.`Scheduled Delivery Date`) + toString(record.

# Run the actual import into the Neo4j DB

In [39]:
# Utility function to write to Neo4j in batch mode.

def insert_data(session, query, frame, batch_size = 500):

    total = 0
    batch = 0
    start = time.time()
    result = None

    while batch * batch_size < len(frame):
        res = session.execute_write( lambda tx: tx.run(query,
                      parameters = {'records': frame[batch*batch_size:(batch+1)*batch_size].to_dict('records')}).data())

        total += res[0]['total']
        batch += 1
        result = {"total":total,
                  "batches":batch,
                  "time":time.time()-start}
        print(result)

    return result

In [40]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth


with GraphDatabase.driver(url, auth=(username, password)) as driver:
    driver.verify_connectivity()

    for c in constraints.keys():
      driver.execute_query(constraints[c])

    session = driver.session(database="neo4j")
    print("Source file: " + data_file_path)
    for q in cypher_import.keys():
      print("importing " + q )
      df = pd.read_csv(data_file_path, encoding = "ISO-8859-1")
      result = insert_data(session, cypher_import[q], df, batch_size = 1000)
      print(result)


Source file: https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session25/resources/csvs/SCMS_Delivery_History_Dataset_20150929.csv
importing Shipment
{'total': 1000, 'batches': 1, 'time': 1.6937587261199951}
{'total': 2000, 'batches': 2, 'time': 2.5980336666107178}
{'total': 3000, 'batches': 3, 'time': 3.485304117202759}
{'total': 4000, 'batches': 4, 'time': 4.4452738761901855}
{'total': 5000, 'batches': 5, 'time': 5.379028797149658}
{'total': 6000, 'batches': 6, 'time': 6.204134941101074}
{'total': 7000, 'batches': 7, 'time': 6.9337639808654785}
{'total': 8000, 'batches': 8, 'time': 7.649659872055054}
{'total': 9000, 'batches': 9, 'time': 8.345618486404419}
{'total': 10000, 'batches': 10, 'time': 9.052983522415161}
{'total': 10324, 'batches': 11, 'time': 9.592599630355835}
{'total': 10324, 'batches': 11, 'time': 9.592599630355835}
importing Product
{'total': 1000, 'batches': 1, 'time': 1.0460126399993896}
{'total': 2000, 'batches': 2, 'time': 1.7030611038208008}
{'total': 3000